In [127]:
import os
import pandas as pd
import multiprocessing as mp
from datetime import datetime, timedelta

In [140]:
def strToFloat(pdf, num_cols):
    for col in num_cols:
        pdf.loc[:, col] = pdf[col].astype(str).str.replace(',', '').astype(float)
    return pdf

def read_csv(filename):
    return pd.read_csv(filename)

In [145]:
data_path = '/Users/fang/StockAnalysis/tmp/data'
common_subfolder = 'common'
bs_subfolder = 'bsReport'

In [131]:
stock_df = pd.read_csv(data_path + os.sep + common_subfolder + os.sep + 'stockList.csv')
stock_df = stock_df[['股票代號', '股票名稱', '市場別']]
stock_df = stock_df.rename(columns={'股票名稱': '標的名稱'})
stock_df.loc[:, '股票代號'] = stock_df['股票代號'].apply(lambda x: str(x))

broker_df = pd.read_csv(data_path + os.sep + common_subfolder + os.sep + 'brokerList.csv')
broker_df.columns = ['券商', '券商名稱', '開業日', '地址', '電話']
broker_df = broker_df[['券商', '券商名稱']]

# cur_dt = datetime.now()
cur_dt = datetime.strptime('20210723', '%Y%m%d')
days = 5
warrant_df_list = []
use_columns = ['標的名稱', '認購認售', '權證代碼', '權證成交量', '權證價格']
for i in range(days):
    data_dt = cur_dt - timedelta(days=i)
    tmp_df = pd.read_csv(os.sep.join([data_path, 'warrant', 'warrant_%s.csv' % data_dt.strftime('%Y-%m-%d')]), usecols=use_columns)
    tmp_df.loc[:, '日期'] = data_dt
    warrant_df_list.append(tmp_df)
    print(data_dt)
warrant_df = pd.concat(warrant_df_list, axis=0, ignore_index=True, sort=False).drop_duplicates()
warrant_not_zero_df = warrant_df[(warrant_df['權證成交量']>0)].copy()
warrant_not_zero_df.loc[:, '成交金額'] = warrant_not_zero_df['權證成交量'] * warrant_not_zero_df['權證價格']
warrant_top100_df = warrant_not_zero_df[['標的名稱', '認購認售', '權證代碼', '成交金額']] \
                                .groupby(['標的名稱','認購認售']) \
                                .agg({'成交金額': sum, '權證代碼': list}) \
                                .reset_index().sort_values('成交金額', ascending=False).head(100)
warrant_top100_df = pd.merge(warrant_top100_df, stock_df, on='標的名稱')

2021-07-23 00:00:00
2021-07-22 00:00:00
2021-07-21 00:00:00
2021-07-20 00:00:00
2021-07-19 00:00:00


In [154]:
top_buyer_list = []
for row in warrant_top100_df.iterrows():
    stock_name = row[1]['標的名稱']
    stock_no = row[1]['股票代號']
    warrant_type = row[1]['認購認售']
    warrant_list = row[1]['權證代碼']
    market_type = 'twse' if row[1]['市場別']=='上市' else 'tpex'
    file_list = []
    print(stock_no, warrant_type)
    count = 0
    print('len(warrant_list):', len(warrant_list))
    for warrant_no in warrant_list:
        for d in range(days):
            data_dt = cur_dt - timedelta(days=d)
            file_path = os.sep.join([data_path, bs_subfolder, data_dt.strftime('%Y%m%d'), market_type, warrant_no + '.csv'])
            if os.path.exists(file_path):
                count += 1
                file_list.append(file_path)
                
    with mp.Pool(processes=2) as pool:
        df_list = pool.map(read_csv, file_list)
        concat_df = pd.concat(df_list, ignore_index=True, sort=False).drop_duplicates()
        concat_df = strToFloat(concat_df.dropna(), ['買進股數', '賣出股數'])

        buy_df = concat_df[concat_df['買進股數'] > 0].copy()
        buy_df.loc[:, '買進金額'] = buy_df['價格'] * buy_df['買進股數']
        buy_df = buy_df.loc[:, ['券商', '買進金額', '買進股數']].groupby("券商").sum().reset_index()
        buy_df.loc[:, '買進均價'] = buy_df['買進金額'] / buy_df['買進股數']
        buy_df = buy_df.dropna()

        sell_df = concat_df[concat_df['賣出股數'] > 0].copy()
        sell_df.loc[:, '賣出金額'] = sell_df['價格'] * sell_df['賣出股數']
        sell_df = sell_df.loc[:, ['券商', '賣出金額', '賣出股數']].groupby("券商").sum().reset_index()
        sell_df.loc[:, '賣出均價'] = sell_df['賣出金額'] / sell_df['賣出股數']
        sell_df = sell_df.dropna()

        merge_df = pd.merge(buy_df, sell_df, on='券商', how='outer').fillna(0.0)
    agg_df = pd.merge(merge_df, broker_df, on='券商')
    agg_df = agg_df[['券商名稱', '買進金額', '買進股數', '賣出金額', '賣出股數']].groupby('券商名稱').sum().reset_index()
    agg_df.loc[:, '買賣超金額'] = (agg_df['買進金額'] - agg_df['賣出金額'])
    total = agg_df['買進金額'].sum()
    agg_df.loc[:, '買超比率'] = agg_df['買賣超金額']/total*100
    agg_df.loc[:, '股票名稱'] = stock_name
    agg_df.loc[:, '股票代號'] = stock_no
    output_cols = ['股票代號', '股票名稱', '券商名稱', '買進金額', '買進股數', '賣出金額', '賣出股數', '買賣超金額', '買超比率']
    display(agg_df[output_cols].sort_values('買超比率', ascending=False).head(3))
    top_buyer_list.append(agg_df[output_cols].sort_values('買超比率', ascending=False).head(3))

2603 認購
len(warrant_list): 862


,股票代號,股票名稱,券商名稱,買進金額,買進股數,賣出金額,賣出股數,買賣超金額,買超比率
660,2603,長榮,統一,135494120.0,5928000.0,30284600.0,3419000.0,105209520.0,9.106668
380,2603,長榮,國票,120568890.0,3077000.0,68798430.0,3247000.0,51770460.0,4.481119
689,2603,長榮,群益金鼎,43801250.0,1809000.0,8438720.0,551000.0,35362530.0,3.060890


2330 認購
len(warrant_list): 1935


,股票代號,股票名稱,券商名稱,買進金額,買進股數,賣出金額,賣出股數,買賣超金額,買超比率
489,2330,台積電,新百王,22131830.0,11249000.0,10818000.0,5698000.0,11313830.0,2.819079
547,2330,台積電,永豐金-信義,14286650.0,7808000.0,7107230.0,3587000.0,7179420.0,1.788903
569,2330,台積電,永豐金-新竹,6955840.0,1525000.0,1321370.0,341000.0,5634470.0,1.403947


2027 認購
len(warrant_list): 795


...

,股票代號,股票名稱,券商名稱,買進金額,買進股數,賣出金額,賣出股數,買賣超金額,買超比率
201,3526,凡甲,永豐金,7698130.0,8456000.0,5361960.0,7063000.0,2336170.0,3.894886
247,3526,凡甲,群益金鼎,7015120.0,6219000.0,5040180.0,4368000.0,1974940.0,3.292640
112,3526,凡甲,凱基-台北,1400690.0,1139000.0,48300.0,30000.0,1352390.0,2.254718


3293 認購
len(warrant_list): 363


,股票代號,股票名稱,券商名稱,買進金額,買進股數,賣出金額,賣出股數,買賣超金額,買超比率
249,3293,鈊象,群益金鼎,13133240.0,9532000.0,8791790.0,6723000.0,4341450.0,7.496114
3,3293,鈊象,中國信託-文心,1078250.0,1200000.0,0.0,0.0,1078250.0,1.861748
238,3293,鈊象,第一金-桃園,756150.0,705000.0,0.0,0.0,756150.0,1.305598


2492 認購
len(warrant_list): 367


,股票代號,股票名稱,券商名稱,買進金額,買進股數,賣出金額,賣出股數,買賣超金額,買超比率
228,2492,華新科,永豐金,10712550.0,10090000.0,8699600.0,8246000.0,2012950.0,3.289468
300,2492,華新科,群益金鼎-敦南,1949740.0,1760000.0,1404770.0,926000.0,544970.0,0.890564
106,2492,華新科,凱基,4877750.0,5744000.0,4414940.0,4766000.0,462810.0,0.756302


In [157]:
result_df = pd.concat(top_buyer_list, ignore_index=True, sort=False).drop_duplicates()
result_df.sort_values('買超比率', ascending=False).to_csv('warrant_key_buyer.csv', index=False)

,券商名稱,買進金額,買進股數,賣出金額,賣出股數,買賣超金額,買超比率,股票代號
29,凱基-竹科,822480.0,649000.0,12300.0,30000.0,810180.0,0.336978,3034
197,富邦,1306800.0,1730000.0,981140.0,1343000.0,325660.0,0.369673,2455
20,元大,22850030.0,29657000.0,21450730.0,28488000.0,1399300.0,0.534376,2317
23,康和-嘉義,2781000.0,1100000.0,1334000.0,500000.0,1447000.0,0.535384,3037
196,元富,5536700.0,4732000.0,5024500.0,4243000.0,512200.0,0.581423,2455
...,...,...,...,...,...,...,...,...
198,群益金鼎,13845940.0,4532000.0,4395320.0,2388000.0,9450620.0,10.915690,3317
273,元大,14103700.0,10174000.0,6685410.0,4720000.0,7418290.0,11.697044,6187
153,元富,17554490.0,10678000.0,3143440.0,1762000.0,14411050.0,12.928585,6756
258,元大,16839000.0,12591000.0,7379500.0,6293000.0,9459500.0,14.117858,3017


In [149]:
row

(96, 標的名稱                                                  華新科
 認購認售                                                   認購
 成交金額                                              63459.2
 權證代碼    [030324, 030831, 031035, 031106, 031125, 03160...
 股票代號                                                 2492
 市場別                                                    上市
 Name: 96, dtype: object)

In [126]:
print(agg_df['買進金額'].sum(), agg_df['賣出金額'].sum())

1155302080.0 1155641380.0


In [54]:
# if len(sys.argv) > 1:
#     dateStr = sys.argv[1]
# else:
#     dateStr = datetime.now().strftime('%Y-%m-%d')
    
dateStr = datetime.now().strftime('%Y-%m-%d')
dateStr = '2021-06-25'
df = pd.read_csv(root_path + os.sep + 'warrant' + os.sep + f'warrant_{dateStr}.csv', dtype={'標的代碼': 'str'})
df['權證成交金額'] = df['權證成交量'] * df['權證價格']
print(df.shape)
df.columns

(27941, 41)


Index(['權證代碼', '權證名稱', '發行券商', '權證價格', '權證漲跌', '權證漲跌幅', '權證成交量', '權證買價',
       '權證賣價', '權證買賣價差', '溢價比率', '價內價外', '理論價格', '隱含波動率', '有效槓桿', '剩餘天數',
       '最新行使比例', '標的代碼', '標的名稱', '標的價格', '標的漲跌', '標的漲跌幅', '最新履約價', '最新界限價',
       '標的20日波動率', '標的60日波動率', '標的120日波動率', '權證DELTA', '權證GAMMA', '權證VEGA',
       '權證THETA', '內含價值', '時間價值', '流通在外估計張數', '流通在外增減張數', '上市日期', '到期日期',
       '最新發行量', '權證發行價', '認購認售', '權證成交金額'],
      dtype='object')

In [55]:
stockInfoDf = pd.read_csv(root_path + os.sep + 'common' + os.sep + 'stockList.csv')
stockInfoDf = stockInfoDf[['股票代號', '市場別']]
stockInfoDf.loc[:, '標的代碼'] = stockInfoDf['股票代號'].apply(lambda x: str(x))

df = pd.merge(df, stockInfoDf[['標的代碼', '市場別']], on='標的代碼')
df.shape

(25650, 42)

In [63]:
df[(df['權證成交量']>50) & (df['市場別']=='上市')].shape

(6625, 42)

In [62]:
df[(df['權證成交量']>50) & (df['市場別']=='上櫃')].shape

(1929, 42)

In [16]:
pd.set_option('display.max_rows', 500)
filteredDf = df[df['權證成交金額'] > 500] 
filteredDf.groupby(['標的代碼', '標的名稱']).agg({'權證成交金額': sum}).sort_values('權證成交金額', ascending=False).head(100)

,,權證成交金額
標的代碼,標的名稱,
2603,長榮,219578.00
2606,裕民,77895.73
2615,萬海,61742.00
2368,金像電,61468.89
3019,亞光,59557.04
2344,華邦電,50160.02
2317,鴻海,47365.63
2027,大成鋼,44571.34
2337,旺宏,42161.53


In [34]:
df[df['標的代碼'] == 6147].sort_values('權證成交金額', ascending=False)[header]

,權證代碼,權證名稱,權證漲跌幅,權證價格,權證成交量,權證成交金額,權證買賣價差,價內價外,剩餘天數,標的價格,流通在外估計張數
23470,719687,頎邦兆豐09購02,-0.0226,1.30,1392,1809.60,0.0078,0.26105,90,70.2,1450
23472,719792,頎邦元大09購02,0.0185,0.55,3017,1659.35,0.0185,0.16923,95,70.2,4242
23459,718788,頎邦統一0A購01,-0.0175,0.56,2837,1588.72,0.0182,0.14181,122,70.2,4076
23481,721325,頎邦元大0B購01,0.0167,1.22,1299,1584.78,0.0083,0.14286,158,70.2,1085
23465,719138,頎邦元大09購01,0.0312,0.66,2124,1401.84,0.0154,0.064,75,70.2,2443
23457,718586,頎邦富邦0A購01,0.0154,0.66,1670,1102.20,0.0154,0.1,118,70.2,2273
23479,721267,頎邦國票0C購01,0.0102,1.98,553,1094.94,0.0051,0.1225,188,70.2,333
23478,721210,頎邦群益12購01,0.0303,2.38,418,994.84,0.0171,0.17412,227,70.2,274
23464,719060,頎邦統一0B購01,0.0333,0.31,2758,854.98,0.0667,0.20227,136,70.2,4984
23477,721019,頎邦凱基0B購01,0.0100,2.02,399,805.98,0.01,0.1225,140,70.2,129


In [33]:
header = ['權證代碼', '權證名稱', '權證漲跌幅', '權證價格', '權證成交量', '權證成交金額', '權證買賣價差',  '價內價外', 
          '剩餘天數', '標的價格','流通在外估計張數']

In [40]:
len(df[df['權證成交量'] > 50]['權證代碼'].unique())

9027

In [42]:
df_list = []
date_list = ['20210614', '20210615', '20210616', '20210617', '20210618', 
             '20210621', '20210622', '20210623', '20210624', '20210625']
for dt in date_list:
    df_list.append(pd.read_csv(root_path + os.sep + 'bsReport' + os.sep + dt + os.sep + 'twse' + os.sep + '033453.csv'))

In [58]:
tmp_df = pd.concat(df_list, axis=0, ignore_index=True, sort=False).drop_duplicates()
tmp_df.columns

Index(['券商', '價格', '買進股數', '賣出股數', '日期'], dtype='object')

In [59]:
tmp_df = strToFloat(tmp_df.copy().dropna(), ['價格', '買進股數', '賣出股數'])
tmp_df['買進金額'] = tmp_df['買進股數'] * tmp_df['價格'] 
tmp_df['賣出金額'] = tmp_df['賣出股數'] * tmp_df['價格'] 
group_df = tmp_df.groupby('券商').agg({'買進股數': sum, '賣出股數': sum, '買進金額': sum, '賣出金額': sum})

In [62]:
group_df['買賣超金額'] = group_df['買進金額'] - group_df['賣出金額']
group_df.sort_values('買賣超金額', ascending=False)

,買進股數,賣出股數,買進金額,賣出金額,買賣超金額
券商,,,,,
8158,3493000.0,0.0,3468050.0,0.0,3468050.0
103F,240000.0,0.0,246050.0,0.0,246050.0
9692,130000.0,30000.0,142900.0,29400.0,113500.0
9659,80000.0,0.0,83800.0,0.0,83800.0
9899,20000.0,0.0,22300.0,0.0,22300.0
9329,20000.0,0.0,22000.0,0.0,22000.0
116K,20000.0,0.0,21200.0,0.0,21200.0
1042,20000.0,0.0,21200.0,0.0,21200.0
116i,15000.0,0.0,16050.0,0.0,16050.0


In [63]:
tmp_df[tmp_df['券商']=='8158']

,券商,價格,買進股數,賣出股數,日期,買進金額,賣出金額
55,8158,1.08,499000.0,0.0,2021/06/17,538920.0,0.0
68,8158,1.06,998000.0,0.0,2021/06/18,1057880.0,0.0
88,8158,0.97,998000.0,0.0,2021/06/21,968060.0,0.0
105,8158,0.90,499000.0,0.0,2021/06/22,449100.0,0.0
106,8158,0.91,499000.0,0.0,2021/06/22,454090.0,0.0


In [108]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [141]:
intro_url = "https://mops.twse.com.tw/mops/web/ajax_t51sb01?encodeURIComponent=1&step=1&firstin=1&TYPEK=sii&code="
download_url  = "https://mops.twse.com.tw/server-java/t105sb02?firstin=true&step=10&filename="
intro_html = requests.post(intro_url).text
soup = BeautifulSoup(intro_html, 'html.parser')

input_list = soup.find_all('input')
download_filename = ''
for i in input_list:
    input_name = i.get('name')
    if input_name == 'filename':
        download_filename = i.get('value')
        
stock_pdf = pd.read_csv(f"{download_url}{download_filename}", encoding='unicode_escape')
stock_list = stock_pdf.iloc[:, 0].to_list()

In [142]:
stock_list

[1101,
 1102,
 1103,
 1104,
 1108,
 1109,
 1110,
 1201,
 1203,
 1210,
 1213,
 1215,
 1216,
 1217,
 1218,
 1219,
 1220,
 1225,
 1227,
 1229,
 1231,
 1232,
 1233,
 1234,
 1235,
 1236,
 1256,
 1301,
 1303,
 1304,
 1305,
 1307,
 1308,
 1309,
 1310,
 1312,
 1313,
 1314,
 1315,
 1316,
 1319,
 1321,
 1323,
 1324,
 1325,
 1326,
 1337,
 1338,
 1339,
 1340,
 1341,
 1342,
 1402,
 1409,
 1410,
 1413,
 1414,
 1416,
 1417,
 1418,
 1419,
 1423,
 1432,
 1434,
 1435,
 1436,
 1437,
 1438,
 1439,
 1440,
 1441,
 1442,
 1443,
 1444,
 1445,
 1446,
 1447,
 1449,
 1451,
 1452,
 1453,
 1454,
 1455,
 1456,
 1457,
 1459,
 1460,
 1463,
 1464,
 1465,
 1466,
 1467,
 1468,
 1470,
 1471,
 1472,
 1473,
 1474,
 1475,
 1476,
 1477,
 1503,
 1504,
 1506,
 1512,
 1513,
 1514,
 1515,
 1516,
 1517,
 1519,
 1521,
 1522,
 1524,
 1525,
 1526,
 1527,
 1528,
 1529,
 1530,
 1531,
 1532,
 1533,
 1535,
 1536,
 1537,
 1538,
 1539,
 1540,
 1541,
 1558,
 1560,
 1563,
 1568,
 1582,
 1583,
 1587,
 1589,
 1590,
 1597,
 1598,
 1603,
 1604,

't51sb01_20250129_173327297.csv'

In [88]:
header = {"Content-Type": "application/x-www-form-urlencoded"}
rsp = requests.post('https://mops.twse.com.tw/server-java/t105sb02?firstin=true&step=10&filename=stock_list.csv', headers=header)

In [36]:
rows[0]

<tr><th align="center" class="tblHead" nowrap="">公司<br/>代號</th><th align="center" class="tblHead" nowrap="">公司名稱</th><th align="center" class="tblHead" nowrap="">公司簡稱</th><th align="center" class="tblHead" nowrap="">產業類別</th><th align="center" class="tblHead" nowrap="">外國企業<br/>註冊地國</th><th align="center" class="tblHead" nowrap="">住址</th><th align="center" class="tblHead" nowrap="">營利事業<br/>統一編號</th><th align="center" class="tblHead" nowrap="">董事長</th><th align="center" class="tblHead" nowrap="">總經理</th><th align="center" class="tblHead" nowrap="">發言人</th><th align="center" class="tblHead" nowrap="">發言人職稱</th><th align="center" class="tblHead" nowrap="">代理發言人</th><th align="center" class="tblHead" nowrap="">總機電話</th><th align="center" class="tblHead" nowrap="">成立日期</th><th align="center" class="tblHead" nowrap="">上櫃日期</th><th align="center" class="tblHead" nowrap="">普通股每股面額</th><th align="center" class="tblHead" nowrap="">實收資本額(元)</th><th align="center" class="tblHead" nowrap="">已發行普通股

[<input name="step" type="hidden" value="0"/>,
 <input name="Market" type="hidden" value="otc"/>,
 <input name="SysName" type="hidden" value="公司彙總報表"/>,
 <input name="reportName" type="hidden" value="公司基本資料查詢彙總表"/>,
 <input name="colorchg" type="hidden" value=""/>,
 <input name="QNum" type="hidden" value="1"/>,
 <input name="Q1N" type="hidden" value="公司類別"/>,
 <input name="Q1V" type="hidden" value="全部"/>,
 <input name="firstin" type="hidden" value="true"/>,
 <input name="step" type="hidden" value="10"/>,
 <input name="filename" type="hidden" value="t51sb01_20250129_172141450.csv"/>,
 <input name="firstin" type="hidden" value="true"/>,
 <input name="step" type="hidden" value="10"/>,
 <input name="filename" type="hidden" value="t51sb01_20250129_172141450.csv"/>]

'1240'

In [143]:
print(f"{len(stock_list)}")

1044


In [166]:
import os 

def getStockList(url):
    intro_url = "https://mops.twse.com.tw/mops/web/ajax_t51sb01?encodeURIComponent=1&step=1&firstin=1&TYPEK=sii&code="
    download_url  = "https://mops.twse.com.tw/server-java/t105sb02?firstin=true&step=10&filename="
    intro_html = requests.post(intro_url).text
    soup = BeautifulSoup(intro_html, 'html.parser')

    input_list = soup.find_all('input')
    download_filename = ''
    for i in input_list:
        input_name = i.get('name')
        if input_name == 'filename':
            download_filename = i.get('value')
            
    stock_pdf = pd.read_csv(f"{download_url}{download_filename}", encoding='unicode_escape')
    stock_list = stock_pdf.iloc[:, 0].apply(str).to_list()
    return sorted(stock_list)

twse_page = 'http://isin.twse.com.tw/isin/C_public.jsp?strMode=2'
stock_list = getStockList(twse_page)

data_path = '/Users/fang/StockAnalysis/data/bs_data/20250129/twse'
stock_file_list = [file_name.replace('.csv', '') for file_name in os.listdir(data_path) if '.csv' in file_name and len(file_name) == 8]
warrant_file_list = [file_name.replace('.csv', '') for file_name in os.listdir(data_path) if '.csv' in file_name and len(file_name) == 10]
file_list = sorted(stock_file_list) + sorted(warrant_file_list)
last_index = 0
if len(file_list) != 0:
    last_index = stock_list.index(sorted(file_list)[-1]) + 1
final_stock_list = stock_list[last_index:]
actual_stock_list = []

In [167]:
final_stock_list

[]

In [256]:
# market: 1=上市; 2=上櫃
# wrn_type: 1=認購; 2=認售
intro_url = "https://mops.twse.com.tw/mops/web/t90sbfa01?encodeURIComponent=1&step=1&ver=1.9&TYPEK=&market=1&wrn_class=all&stock_no=&wrn_no=&co_id=all&wrn_type=1&left_month=all&return_rate=all&price_down=&price_up=&price_inout=all&newprice_down=&newprice_up=&fin_down=&fin_up=&sort=1"
download_url  = "https://mops.twse.com.tw/server-java/t105sb02?step=10&ver=1.9&TYPEK=&market=1&wrn_class=2&stock_no=&wrn_no=&co_id=all&wrn_type=all&left_month=all&return_rate=all&price_down=&price_up=&price_inout=all&newprice_down=&newprice_up=&fin_down=&fin_up=&sort=1&firstin=1&isShowForm=1&pageno=&pagesize=&filename="
intro_html = requests.post(intro_url).text
soup = BeautifulSoup(intro_html, 'html.parser')

input_list = soup.find_all('input')
download_filename = ''
for i in input_list:
    input_name = i.get('name')
    if input_name == 'filename':
        download_filename = i.get('value')

warrant_pdf = pd.read_csv(f"{download_url}{download_filename}", encoding='big5hkscs')
warrant_list = warrant_pdf.iloc[:, 0].str.replace("=", "").str.replace("\"", "").to_list()

In [287]:
#!/userap/anaconda3/bin/python3
# -*- coding: utf-8 -*-
import os
import cv2
import shutil
import logging
import requests
import operator
import traceback
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from io import StringIO
from os import listdir
from time import sleep, time
from random import randint
from datetime import datetime, timedelta
from bs4 import BeautifulSoup

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logging.info('Start time: {}'.format(datetime.now()))
start = time()

''' 
# this version of getStockList only includes stocks but not warrants
def getStockList(url):
    df = pd.read_html(url,encoding='big5hkscs',header=0)[0]
    raw_list = df['有價證券代號及名稱']
    code_list = []
    for code in raw_list:
        code = code.split('　')[0]
        if len(code) == 4:
            code_list.append(code)
    return code_list
'''

def getStockList():
    intro_url = "https://mops.twse.com.tw/mops/web/ajax_t51sb01?encodeURIComponent=1&step=1&firstin=1&TYPEK=sii&code="
    download_url  = "https://mops.twse.com.tw/server-java/t105sb02?firstin=true&step=10&filename="
    intro_html = requests.post(intro_url).text
    soup = BeautifulSoup(intro_html, 'html.parser')

    input_list = soup.find_all('input')
    download_filename = ''
    for i in input_list:
        input_name = i.get('name')
        if input_name == 'filename':
            download_filename = i.get('value')
            
    stock_pdf = pd.read_csv(f"{download_url}{download_filename}", encoding='unicode_escape')
    stock_list = stock_pdf.iloc[:, 0].apply(str).to_list()
    return sorted(stock_list)


def getWarrantList():
    # market: 1=上市; 2=上櫃 / wrn_class: 1=認購; 2=認售
    wrn_classes = ['1', '2']
    intro_url = "https://mops.twse.com.tw/mops/web/t90sbfa01?encodeURIComponent=1&step=1&ver=1.9&TYPEK=&market=1&wrn_type=all&stock_no=&wrn_no=&co_id=all&left_month=all&return_rate=all&price_down=&price_up=&price_inout=all&newprice_down=&newprice_up=&fin_down=&fin_up=&sort=1&wrn_class="
    download_url  = "https://mops.twse.com.tw/server-java/t105sb02?firstin=true&step=10&filename="

    wrn_list = []
    for w_class in wrn_classes:
        intro_html = requests.post(f"{intro_url}{w_class}").text
        soup = BeautifulSoup(intro_html, 'html.parser')
        input_list = soup.find_all('input')
        download_filename = ''
        for i in input_list:
            input_name = i.get('name')
            if input_name == 'filename':
                download_filename = i.get('value')
        wrn_pdf = pd.read_csv(f"{download_url}{download_filename}", encoding='big5hkscs')
        wrn_list += wrn_pdf.iloc[:, 0].str.replace("=", "").str.replace("\"", "").to_list()
        print(len(wrn_list))
    return wrn_list

def mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float"))**2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

def getMseTuples(pic):
    mse_dict = {}
    for alphabet in alphabet_dict:
        mse_dict[alphabet] = mse(alphabet_dict[alphabet], pic)
    sorted_mse = sorted(mse_dict.items(), key=operator.itemgetter(1))
    return sorted_mse

def captchaSolver(img_path):
    image = cv2.imread(img_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, threshold = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
    kernel = np.ones((2,2), np.uint16)
    erosion = cv2.erode(threshold, kernel, iterations=2)
    blurred = cv2.GaussianBlur(erosion, (3,3), 0)
    contours, hierarchy = cv2.findContours(blurred.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted([(c, cv2.boundingRect(c)[0]) for c in contours], key = lambda x: x[1]) 
    
    ary = []
    for (c, _) in cnts:
        (x, y, w, h) = cv2.boundingRect(c)
        if w>15 and h>15:
            ary.append((x, y, w, h))      

    index = 0
    prev = (-100, -100, 100, 100)
    remove_list = []
    for _, obj in enumerate(ary):
        if (obj[0]-prev[0] <= 15
            and obj[1]-prev[1] <= 15
            and obj[2] < prev[2]
            and obj[3] < prev[3]):
            remove_list.append(obj)
        prev = obj
    
    for obj in remove_list:
        ary.remove(obj)
        
    if len(ary) != 5:
        return None
    
    result = ''
    for id, (x, y, w, h) in enumerate(ary):
        roi = blurred[y:y+h, x:x+w]
        thresh = roi.copy()
        res = cv2.resize(thresh, (50, 50))
        kernel = np.ones((3,3), np.uint16)
        edged = cv2.Canny(res, 100, 200)
        res = cv2.dilate(edged, kernel, iterations=2)
        
        mse_dict = getMseTuples(res)
        first = mse_dict[0]
        second = mse_dict[1]
        if second[1]-first[1] <= 1000:
            return None
        if first[1] > 18000:
            return None
            
        result += first[0]
    return result


logging.info("Loading alphabet pictures for captchaSolver")
root_path = '/Users/fang/StockAnalysis'
alphabet_dict = {}
for png in os.listdir(root_path + os.sep + "img" + os.sep + 'alphabet'):
    if '.png' not in png:
        continue
    alphabet = png.replace('.png', '')

    image = ~(cv2.imread(root_path + os.sep + "img" + os.sep + 'alphabet' + os.sep + png))

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, threshold = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
    
    kernel = np.ones((3,3), np.uint16)
    erosion = cv2.dilate(threshold, kernel, iterations=1)
    
    contours, hierarchy = cv2.findContours(erosion.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted([(c, cv2.boundingRect(c)[0]) for c in contours], key = lambda x: x[1])

    for (c, _) in cnts:
        (x, y, w, h) = cv2.boundingRect(c)
        contour = erosion[y:y+h, x:x+w]
        res = cv2.resize(contour, (50, 50))
        ret, threshold = cv2.threshold(res, 200, 255, cv2.THRESH_BINARY)
        alphabet_dict[alphabet] = threshold
        break

session = None
root_url = 'https://bsr.twse.com.tw/bshtm/'
twse_page = 'http://isin.twse.com.tw/isin/C_public.jsp?strMode=2'

logging.info('get stock list')
stock_list = getWarrantList()

today_dt = datetime.now()
hour = today_dt.hour
if hour < 9:
    today_dt = today_dt - timedelta(hours=9)
data_dt = today_dt.strftime('%Y%m%d')

logging.info('Check if data path is not existed')
data_path = root_path + os.sep + 'data' + os.sep + 'bs_data' + os.sep + data_dt 
if not os.path.exists(data_path + os.sep + 'twse'):
    # check its parent folder
    if not os.path.exists(data_path):
        os.mkdir(data_path)
    os.mkdir(data_path + os.sep + 'twse')
data_path = data_path + os.sep + 'twse'

logging.info('Check if any stocks/warrants had been crawled')
stock_file_list = [file_name.replace('.csv', '') for file_name in os.listdir(data_path) if '.csv' in file_name and len(file_name) == 8]
warrant_file_list = [file_name.replace('.csv', '') for file_name in os.listdir(data_path) if '.csv' in file_name and len(file_name) == 10]
file_list = sorted(stock_file_list) + sorted(warrant_file_list)
last_index = 0
if len(file_list) != 0:
    last_index = stock_list.index(sorted(file_list)[-1]) + 1
final_stock_list = stock_list[last_index:]
actual_stock_list = []

logging.info('Start to crawl data')
for stock_code in final_stock_list:
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36' }
    failed = True
    failed_cnt = 0
    logging.info(f"Trying to crawl data: {stock_code}")
    while failed:
        try:
            if session != None:
                session.close()
            session = requests.Session()
            response = session.get(root_url + 'bsMenu.aspx', headers=headers)
            if response.ok:
                soup = BeautifulSoup(response.text, 'html.parser')
                hidden_elements = soup.find_all('input', type='hidden')

                # get captcha image to solve
                img_id = soup.find_all('img')[1].attrs['src']
                img_url = root_url + img_id
                img = session.get(img_url, stream = True)
                if img.ok:
                    logging.debug('download captcha image')
                    f = open(root_path + os.sep + 'tmp.png', 'wb')
                    shutil.copyfileobj(img.raw, f)
                    f.close()

                    logging.debug('solve captcha by image recognition')
                    captcha_num = captchaSolver(root_path + os.sep + 'tmp.png')
                    if captcha_num == None:
                        logging.warning('captcha failed')
                        failed_cnt += 1
                        continue

                    logging.debug('captch pass')
                    logging.debug('send post request to get data')
                    payload = {'RadioButton_Normal':'RadioButton_Normal', 
                               'TextBox_Stkno': stock_code, 
                               'btnOK':'查詢',
                               'CaptchaControl1': captcha_num}
                    for element in hidden_elements:
                        payload[element.attrs['name']] = element.attrs['value']
                    post_response = session.post(root_url + 'bsMenu.aspx', data = payload, headers = headers)
                    if '查無資料' in post_response.text:
                        logging.info('no result found.')
                        open(root_path + os.sep + "data" + os.sep + 'bs_data' + os.sep + data_dt + os.sep + 'twse' + os.sep + '{}.csv'.format(stock_code), "a")
                        failed = False
                        continue
                    if 'HyperLink_DownloadCSV' not in post_response.text:
                        continue

                    logging.debug('extract transaction date from html page')
                    bs_page = session.get(root_url + 'bsContent.aspx?v=t', headers = headers)
                    html_content = BeautifulSoup(bs_page.text, 'html.parser')
                    main_table = html_content.select_one('table table table')
                    if not main_table:
                        failed_cnt += 1
                        logging.warning('main_table is None')
                        continue
                    tx_date = main_table.select('tr td')[1].text.strip().replace('\r\n', '')

                    logging.debug('parse and download buy/sell report data')
                    bs_report = session.get(root_url + 'bsContent.aspx', headers = headers)
                    if not bs_report.ok:
                        failed += 1
                        logging.warning('403 forbidden while crawling data')
                        continue
                    bs_df = pd.read_csv(StringIO(bs_report.text), sep=',', skiprows=2)
                    df_part1 = bs_df.iloc[:, :5]
                    df_part2 = bs_df.iloc[:, 6:]
                    df_part2.columns = df_part1.columns
                    df_merged = pd.concat([df_part1,df_part2], axis=0).sort_values('序號')
                    df_merged['券商'] = df_merged['券商'].str[:4]
                    del df_merged['序號']
                    df_merged['日期'] = tx_date
                    df_merged.to_csv(root_path + os.sep + "data" + os.sep + 'bs_data' + os.sep + data_dt + os.sep + 'twse' + os.sep + '{}.csv'.format(stock_code), encoding='utf8', index=False)
                    actual_stock_list.append(stock_code)
                    logging.info(f'{stock_code} has been crawled with {failed_cnt} failures')
                    failed = False
        except Exception as e:
            logging.warning(traceback.format_exc())
end = time()
logging.info(f"Total count: {len(final_stock_list)}")
logging.info(f"Actual count: {len(set(actual_stock_list))}")
logging.info(f"Missed stock: {set(final_stock_list)-set(actual_stock_list)}")
logging.info(f"Execution Time:{end-start}")
logging.info(f"End time: {datetime.now()}")

2025-02-02 13:27:24,003 [INFO] Start time: 2025-02-02 13:27:24.003109
2025-02-02 13:27:24,005 [INFO] Loading alphabet pictures for captchaSolver
2025-02-02 13:27:24,058 [INFO] get stock list


31163


2025-02-02 13:27:31,054 [INFO] Check if data path is not existed
2025-02-02 13:27:31,056 [INFO] Check if any stocks/warrants had been crawled
2025-02-02 13:27:31,061 [INFO] Start to crawl data
2025-02-02 13:27:31,063 [INFO] Trying to crawl data: 031489
2025-02-02 13:27:31,253 [INFO] 031489 has been crawled with 0 failures
2025-02-02 13:27:31,254 [INFO] Trying to crawl data: 031490


34887


2025-02-02 13:27:31,372 [WARNING] captcha failed
2025-02-02 13:27:31,475 [WARNING] captcha failed
2025-02-02 13:27:31,589 [INFO] no result found.
2025-02-02 13:27:31,590 [INFO] Trying to crawl data: 031491
2025-02-02 13:27:31,693 [WARNING] captcha failed
2025-02-02 13:27:31,869 [INFO] 031491 has been crawled with 1 failures
2025-02-02 13:27:31,870 [INFO] Trying to crawl data: 031494
2025-02-02 13:27:31,971 [WARNING] captcha failed
2025-02-02 13:27:32,112 [INFO] no result found.
2025-02-02 13:27:32,122 [INFO] Trying to crawl data: 031497
2025-02-02 13:27:32,295 [INFO] 031497 has been crawled with 0 failures
2025-02-02 13:27:32,296 [INFO] Trying to crawl data: 031520
2025-02-02 13:27:41,277 [INFO] 031520 has been crawled with 0 failures
2025-02-02 13:27:41,285 [INFO] Trying to crawl data: 031523
2025-02-02 13:27:41,406 [WARNING] captcha failed
2025-02-02 13:27:41,538 [WARNING] captcha failed
2025-02-02 13:27:41,659 [WARNING] captcha failed
2025-02-02 13:27:41,858 [WARNING] captcha failed

...

2025-02-02 21:26:36,716 [INFO] Trying to crawl data: 045219
2025-02-02 21:26:36,809 [WARNING] captcha failed
2025-02-02 21:26:36,925 [INFO] no result found.
2025-02-02 21:26:36,927 [INFO] Trying to crawl data: 045220
2025-02-02 21:26:37,051 [INFO] no result found.
2025-02-02 21:26:37,054 [INFO] Trying to crawl data: 045221
2025-02-02 21:26:37,159 [WARNING] captcha failed
2025-02-02 21:26:37,276 [INFO] no result found.
2025-02-02 21:26:37,277 [INFO] Trying to crawl data: 045222
2025-02-02 21:26:37,417 [INFO] no result found.
2025-02-02 21:26:37,419 [INFO] Trying to crawl data: 045223
2025-02-02 21:26:37,544 [WARNING] captcha failed
2025-02-02 21:26:46,452 [WARNING] captcha failed
2025-02-02 21:26:46,564 [INFO] no result found.
2025-02-02 21:26:46,567 [INFO] Trying to crawl data: 045224
2025-02-02 21:26:46,681 [INFO] no result found.
2025-02-02 21:26:46,684 [INFO] Trying to crawl data: 045225
2025-02-02 21:26:46,777 [WARNING] captcha failed
2025-02-02 21:26:46,865 [WARNING] captcha faile

2025-02-02 21:27:59,730 [WARNING] captcha failed
2025-02-02 21:27:59,823 [WARNING] captcha failed
2025-02-02 21:27:59,985 [WARNING] captcha failed
2025-02-02 21:28:08,422 [INFO] no result found.
2025-02-02 21:28:08,424 [INFO] Trying to crawl data: 045272
2025-02-02 21:28:08,522 [WARNING] captcha failed
2025-02-02 21:28:08,687 [INFO] 045272 has been crawled with 1 failures
2025-02-02 21:28:08,688 [INFO] Trying to crawl data: 045273
2025-02-02 21:28:08,868 [INFO] 045273 has been crawled with 0 failures
2025-02-02 21:28:08,870 [INFO] Trying to crawl data: 045274
2025-02-02 21:28:09,103 [INFO] no result found.
2025-02-02 21:28:09,107 [INFO] Trying to crawl data: 045275
2025-02-02 21:28:09,215 [WARNING] captcha failed
2025-02-02 21:28:09,327 [WARNING] captcha failed
2025-02-02 21:28:09,505 [INFO] no result found.
2025-02-02 21:28:09,506 [INFO] Trying to crawl data: 045276
2025-02-02 21:28:09,633 [WARNING] captcha failed
2025-02-02 21:28:18,434 [WARNING] captcha failed
2025-02-02 21:28:18,61

2025-02-02 21:44:26,558 [INFO] no result found.
2025-02-02 21:44:26,560 [INFO] Trying to crawl data: 045304
2025-02-02 21:44:35,611 [INFO] no result found.
2025-02-02 21:44:35,614 [INFO] Trying to crawl data: 045305
2025-02-02 21:44:35,797 [INFO] 045305 has been crawled with 0 failures
2025-02-02 21:44:35,799 [INFO] Trying to crawl data: 045306
2025-02-02 21:44:35,967 [INFO] 045306 has been crawled with 0 failures
2025-02-02 21:44:35,968 [INFO] Trying to crawl data: 045307
2025-02-02 21:44:36,138 [INFO] no result found.
2025-02-02 21:44:36,140 [INFO] Trying to crawl data: 045308
2025-02-02 21:44:36,292 [INFO] no result found.
2025-02-02 21:44:36,294 [INFO] Trying to crawl data: 045309
2025-02-02 21:44:36,395 [WARNING] captcha failed
2025-02-02 21:44:36,532 [INFO] no result found.
2025-02-02 21:44:36,535 [INFO] Trying to crawl data: 045310
2025-02-02 21:44:36,762 [INFO] 045310 has been crawled with 0 failures
2025-02-02 21:44:36,763 [INFO] Trying to crawl data: 045311
2025-02-02 21:44:4

2025-02-02 22:02:52,396 [INFO] 045360 has been crawled with 0 failures
2025-02-02 22:02:52,397 [INFO] Trying to crawl data: 045361
2025-02-02 22:02:52,511 [INFO] no result found.
2025-02-02 22:02:52,513 [INFO] Trying to crawl data: 045362
2025-02-02 22:02:52,602 [WARNING] captcha failed
2025-02-02 22:02:52,716 [INFO] no result found.
2025-02-02 22:02:52,719 [INFO] Trying to crawl data: 045363
2025-02-02 22:02:52,950 [INFO] no result found.
2025-02-02 22:02:52,951 [INFO] Trying to crawl data: 045364
2025-02-02 22:02:53,095 [INFO] no result found.
2025-02-02 22:02:53,097 [INFO] Trying to crawl data: 045365
2025-02-02 22:02:53,190 [WARNING] captcha failed
2025-02-02 22:02:53,407 [WARNING] captcha failed
2025-02-02 22:03:02,399 [INFO] no result found.
2025-02-02 22:03:02,401 [INFO] Trying to crawl data: 045366
2025-02-02 22:03:02,515 [WARNING] captcha failed
2025-02-02 22:03:02,636 [INFO] no result found.
2025-02-02 22:03:02,638 [INFO] Trying to crawl data: 045367
2025-02-02 22:03:02,738 [

KeyboardInterrupt: 

In [282]:
bs_report.status_code

403

In [368]:
import json

pdf_list = []
# call: 0999 / put: 0999P
warrant_type = ['0999', '0999P']
for w_type in warrant_type:
    resp = requests.get(f'https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date=20250214&type={w_type}&response=json')
    obj = json.loads(resp.content.decode('utf8'))
    table = None
    for t in obj['tables']:
        if len(t) != 0:
            table = t
    fields = table['fields']
    data = table['data']
    pdf = pd.DataFrame(data, columns=fields)
    pdf['成交股數'] = pd.to_numeric(pdf['成交股數'].str.replace(',', ''), errors='coerce')
    pdf['成交筆數'] = pd.to_numeric(pdf['成交筆數'].str.replace(',', ''), errors='coerce')
    pdf = pdf[pdf['成交筆數']!=0]
    pdf_list.append(pdf)
concat_pdf = pd.concat(pdf_list, axis=0)
concat_pdf = concat_pdf.sort_values('成交股數', ascending=False)
pdf[]

,暫停交易,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,標的代號,標的名稱,標的收盤價/指數
12445,,053237,中鋼元大44購01,7186000,50,"829,490",0.07,0.14,0.07,0.12,<p style= color:red>+</p>,0.05,0.12,499,0.13,10,23.25,2002,中鋼,23.25
733,,04399P,臺股指群益44售15,6351000,28,"1,080,520",0.17,0.19,0.17,0.18,<p style= color:red>+</p>,0.02,0.17,499,0.18,500,0.00,IX0001,臺股指數,"23,152.61"
17562,,058397,群創中信45購01,5897000,66,"2,554,850",0.44,0.46,0.40,0.46,<p style= color:red>+</p>,0.04,0.46,9,0.47,10,0.00,3481,群創,14.15
20154,,061018,華新元大46購01,5135000,56,"5,336,380",0.79,1.55,0.79,1.55,<p style= color:red>+</p>,0.70,1.50,5,1.55,19,28.75,1605,華新,28.75
26749,,067948,華邦電永豐48購01,4999000,42,"9,504,590",1.74,2.06,1.74,2.03,<p style= color:red>+</p>,0.72,2.10,499,--,0,17.15,2344,華邦電,17.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12733,,053526,亞光群益44購02,0,0,0,--,--,--,--,<p> </p>,0.00,--,0,--,0,195.00,3019,亞光,195.00
12729,,053522,盟立群益46購01,0,0,0,--,--,--,--,<p> </p>,0.00,0.28,10,--,0,78.40,2464,盟立,78.40
12728,,053521,旺宏群益44購04,0,0,0,--,--,--,--,<p> </p>,0.00,--,0,--,0,21.10,2337,旺宏,21.10
12727,,053520,台積電群益47購17,0,0,0,--,--,--,--,<p> </p>,0.00,0.78,100,0.80,499,"1,060.00",2330,台積電,"1,060.00"


In [371]:
pdf[pdf['成交筆數']!=0]['證券代號'].to_list()

['053237',
 '04399P',
 '058397',
 '061018',
 '067948',
 '030439',
 '058624',
 '062324',
 '05775P',
 '063753',
 '036200',
 '064732',
 '065641',
 '046722',
 '067127',
 '067459',
 '052820',
 '052778',
 '031950',
 '059508',
 '043062',
 '055433',
 '045106',
 '04929P',
 '052838',
 '067988',
 '05806P',
 '063608',
 '058689',
 '067736',
 '06019P',
 '069668',
 '036598',
 '067601',
 '054263',
 '064276',
 '061993',
 '031241',
 '047780',
 '06295P',
 '06971P',
 '070111',
 '05579P',
 '051699',
 '083628',
 '060917',
 '039509',
 '053484',
 '040535',
 '052626',
 '052049',
 '053177',
 '035979',
 '059194',
 '057954',
 '056825',
 '042900',
 '065615',
 '052373',
 '065650',
 '061750',
 '050129',
 '055161',
 '042344',
 '060596',
 '040188',
 '033616',
 '066758',
 '063193',
 '065134',
 '036961',
 '056580',
 '057890',
 '057922',
 '062760',
 '05559P',
 '07647T',
 '058722',
 '068377',
 '063910',
 '06555P',
 '050125',
 '035297',
 '060008',
 '037452',
 '033518',
 '042866',
 '065886',
 '040749',
 '063449',
 '064978',

In [372]:
from datetime import datetime

In [387]:
today = datetime.now().strftime('%Y%m%d')
resp = requests.get(f'https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date={today}&type=0099P&response=json')
obj = json.loads(resp.content.decode('utf8'))
table = None
for t in obj['tables']:
    if len(t) != 0:
        table = t
fields = table['fields']
data = table['data']
pdf = pd.DataFrame(data, columns=fields)
etf_list = pdf['證券代號'].to_list()

resp = requests.get(f'https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date={today}&type=ALLBUT0999&response=json')
obj = json.loads(resp.content.decode('utf8'))
table = None
for t in obj['tables']:
    if len(t) != 0:
        table = t
fields = table['fields']
data = table['data']
pdf = pd.DataFrame(data, columns=fields)
pdf = pdf[pdf['成交筆數']!=0]
pdf = pdf[~pdf['證券代號'].isin(etf_list)]
stock_list = pdf[pdf['證券代號'].apply(lambda x: len(x) == 4)]['證券代號'].to_list()



In [390]:
stock_list

['1101',
 '1102',
 '1103',
 '1104',
 '1108',
 '1109',
 '1110',
 '1201',
 '1203',
 '1210',
 '1213',
 '1215',
 '1216',
 '1217',
 '1218',
 '1219',
 '1220',
 '1225',
 '1227',
 '1229',
 '1231',
 '1232',
 '1233',
 '1234',
 '1235',
 '1236',
 '1256',
 '1301',
 '1303',
 '1304',
 '1305',
 '1307',
 '1308',
 '1309',
 '1310',
 '1312',
 '1313',
 '1314',
 '1315',
 '1316',
 '1319',
 '1321',
 '1323',
 '1324',
 '1325',
 '1326',
 '1337',
 '1338',
 '1339',
 '1340',
 '1341',
 '1342',
 '1402',
 '1409',
 '1410',
 '1413',
 '1414',
 '1416',
 '1417',
 '1418',
 '1419',
 '1423',
 '1432',
 '1434',
 '1435',
 '1436',
 '1437',
 '1438',
 '1439',
 '1440',
 '1441',
 '1442',
 '1443',
 '1444',
 '1445',
 '1446',
 '1447',
 '1449',
 '1451',
 '1452',
 '1453',
 '1454',
 '1455',
 '1456',
 '1457',
 '1459',
 '1460',
 '1463',
 '1464',
 '1465',
 '1466',
 '1467',
 '1468',
 '1470',
 '1471',
 '1472',
 '1473',
 '1474',
 '1475',
 '1476',
 '1477',
 '1503',
 '1504',
 '1506',
 '1512',
 '1513',
 '1514',
 '1515',
 '1516',
 '1517',
 '1519',
 

In [391]:
today_dt = datetime.now()
#     if today_dt.hour .strftime('%Y%m%d')

In [392]:
today_dt.hour

14

In [393]:
today = datetime.now().strftime('%Y%m%d')
resp = requests.get(f'https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date={today}&type=0099P&response=json')
obj = json.loads(resp.content.decode('utf8'))
table = None
for t in obj['tables']:
    if len(t) != 0:
        table = t
fields = table['fields']
data = table['data']
pdf = pd.DataFrame(data, columns=fields)
etf_list = pdf['證券代號'].to_list()

In [394]:
etf_list

['0050',
 '0051',
 '0052',
 '0053',
 '0055',
 '0056',
 '0057',
 '0061',
 '006203',
 '006204',
 '006205',
 '006206',
 '006207',
 '006208',
 '00625K',
 '00631L',
 '00632R',
 '00633L',
 '00634R',
 '00635U',
 '00636',
 '00636K',
 '00637L',
 '00638R',
 '00639',
 '00640L',
 '00641R',
 '00642U',
 '00643',
 '00643K',
 '00645',
 '00646',
 '00647L',
 '00648R',
 '00650L',
 '00651R',
 '00652',
 '00653L',
 '00654R',
 '00655L',
 '00656R',
 '00657',
 '00657K',
 '00660',
 '00661',
 '00662',
 '00663L',
 '00664R',
 '00665L',
 '00666R',
 '00668',
 '00668K',
 '00669R',
 '00670L',
 '00671R',
 '00673R',
 '00674R',
 '00675L',
 '00676R',
 '00678',
 '00680L',
 '00681R',
 '00682U',
 '00683L',
 '00684R',
 '00685L',
 '00686R',
 '00688L',
 '00689R',
 '00690',
 '00692',
 '00693U',
 '00700',
 '00701',
 '00702',
 '00703',
 '00706L',
 '00707R',
 '00708L',
 '00709',
 '00710B',
 '00711B',
 '00712',
 '00713',
 '00714',
 '00715L',
 '00717',
 '00728',
 '00730',
 '00731',
 '00733',
 '00735',
 '00736',
 '00737',
 '00738U',
 

In [395]:
resp = requests.get(f'https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date={today}&type=ALLBUT0999&response=json')
obj = json.loads(resp.content.decode('utf8'))
table = None
for t in obj['tables']:
    if len(t) != 0:
        table = t
fields = table['fields']
data = table['data']
pdf = pd.DataFrame(data, columns=fields)
pdf = pdf[pdf['成交筆數']!=0]
pdf = pdf[~pdf['證券代號'].isin(etf_list)]

In [396]:
from datetime import timedelta

In [400]:
datetime.now() - timedelta(days=1)

datetime.datetime(2025, 3, 6, 14, 56, 49, 260454)